<a href="https://githubtocolab.com/minyeamer/groove_midi/blob/main/notebooks/background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

## Explore Background
- Find prior knowledge for VAE
- Review suggested [reference](https://arxiv.org/pdf/1803.05428.pdf)
- Search additional information for MusicVAE

<hr style="border-style:dotted">

### VAE(Auto-Encoding Variational Bayes)

<img src="https://user-images.githubusercontent.com/24144491/50323466-18d03700-051d-11e9-82ed-afb1b6e2666a.png" width="50%">

- VAE는, Encoder로 차원축소를 하기 위한 목적의 Audoencoder와 반대로, Decoder로 새로운 데이터를 생성하기 위한 목적을 가집니다.
- 입력 데이터 $x$로부터 인코더를 통해 추출한 특징에 대한 latent vector $z$를, 다시 디코더를 거쳐 $x$와 유사한 데이터로 생성해내는 과정을 거칩니다.
- 우선, $x_i$에 대해 Gaussian Encoder를 통과한 출력 결과인 $\mu, \sigma$를 가지고 정규 분포를 생성하며, 해당 분포에서 $z_i$를 샘플링합니다.
- 다음으로, $z_i$를 가지고 Bernoulli Decoder를 통과하면 $x_i$와 동일한 출력 결과 $p_i$을 생성합니다.
- 샘플링에 대한 역전파 과정을 수행하기 위해 Reparameterization Trick을 사용합니다.   
  ($N(0,1)$을 따르는 정규 분포 $\epsilon_i$에 $\sigma_i^2$를 곱하고 $\mu_i$를 더하는 과정을 통해 backward를 저장하게 됩니다.)
- latent vector로부터 가장 그럴듯한 확률 분포를 찾아내기 때문에, Decoder가 생성한 데이터가 학습 데이터와 닮은 특징을 가집니다.

<br>

#### Backpropagation을 위한 Sampling 과정

```python
import keras.backend as K

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0],latent_dim),mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])
```

<br>

#### Loss Function

<img src="https://user-images.githubusercontent.com/24144491/50323472-1a016400-051d-11e9-86b7-d8bf6a1a880f.png" width="50%">

- Loss Function은 Reconstruction Error(복원 오차)와 Regularization 과정에서 도출되는 KL divergence(제약 조건)의 합으로 구해집니다.
- Reconstruction Error는 확률을 정규 분포로 가정하면 MSE, 이항 분포로 가정하면 CrossEntropy로 정의됩니다. (일반적으론 베르누이로 지정합니다.)
- Regularization은 인코더를 통과해 나온 확률 분포가 정규 분포로 가정된 $p(z)$를 따르도록 하기 위해 두 분포 간 거리(KL divergence)를 최소화하는 과정입니다.
- 증명 과정을 통해 도출된 likelihood에 대한 ELBO 식 $\log{p_{\theta}(x^{(i)})\ge{L(x^{(i)},\theta,\phi)}}$에서 lower bound인 Loss Function을 maximize하는,   
  디코더의 파라미터 $\theta$와 인코더의 파라미터 $\phi$를 찾기 위함이 주 목적입니다.
- 최적화 과정에서는 Loss Function을 최소화하기 위해 Reconstruction Error에 마이너스를 곱해서 계산합니다.

<br>

#### Loss Function 도출 과정

```python
import keras.backend as K

def vae_loss(self, x, z_decoded):
    x = K.flatten(x)
    z_decoded = K.flatten(z_decoded)
    xent_loss = keras.metrics.binary_crossentropy(x,z_decoded)
    kl_loss   = -5e-4*K.mean(1+z_log_var-K.square(z_mean)-K.exp(z_log_var),axis=-1)
    return K.mean(xent_loss + kl_loss)
```

<br>

#### References
- [KAIST 스마트설계연구실 강남우 교수(전 숙명여대)의 딥러닝과 설계 강의](https://youtu.be/GbCAwVVKaHY)
- [VAE(Auto-Encoding Variational Bayes) 직관적 이해](https://taeu.github.io/paper/deeplearning-paper-vae/)

<hr style="border-style:dotted">

### A Hierarchical Latent Vector Model for Learning Long-Term Structure in Music

#### 1. Introduction
- 생성 모델링은 데이터 x를 생성하기 위한 확률 분포 $p(x)$를 도출하는 과정입니다.
- 최근(2018)의 생성 모델링 연구는 복잡한 구조를 사용하는 DNN을 통해 가속화되었으며,   
  해당 연구에서는 VAE 등 deep latent variable 모델에 주목합니다.
- 여전히 긴 시퀀스에 대해 deep latent variable 모델은 성과를 내지 못하고 있는데,   
  이를 보완하기 위해 계층적 순환 디코더를 포함하는 오토인코더를 제안합니다.
- 제안할 모델은 전체 latent vector를 인코딩하여, baseline인 flat decoder RNN보다 긴 시퀀스에 효과적입니다.

#### 2. Background

##### 2-2. Reccurent VAEs

<img src="https://i.imgur.com/PQKoraX.png" width="50%">

- 인코더는 입력 시퀀스에 대한 hidden states를 생성하는 RNN 모델입니다.
- latent code $z$의 분포는 $h_T$로 설정되며, 동시에 디코더 RNN의 초기 상태로 사용됩니다.
- 출력 시퀀스 $y$를 발생시키는 디코더는 일반적인 VAE처럼 $p(z)$에 근사해지는 $q_{\lambda}(z|x)$를 학습합니다.

#### 3. Model

##### 3.1. Bidirectional Encoder
- 인코더에 대해 2계층 양방향 LSTM 구조를 사용하여 마지막 상태 벡터 $\overrightarrow{h_T}, \overleftarrow{h_T}$를 도출합니다.
- 두 벡터를 결합한 $h_T$를 2개의 fully-connected 레이어에 넣어서 $\mu$와 $\sigma$를 생성합니다.   
  ($\ \mu=W_{h\mu}h_T+b_\mu,\ \sigma=\log{(\exp{(W_{h\sigma}h_T+b_{\sigma})+1})}\ $)
- 모든 레이어에 대해 2048개의 state size와 512 latent dimension을 가지는 LSTM이 입력 시퀀스에 대한 문맥으로부터 $\mu$와 $\sigma$를 도출하게 됩니다.

##### 3.2. Hierachical Decoder
- 2계층 디코더 RNN에 대해 각각의 계층에서 1024 units으로 구성된 LSTM을 사용합니다.
- 디코더 RNN은 초기 상태로 latent vector $z$가 설정되고, autoregressive하게 출력 시퀀스를 생성합니다.
- 하지만, 이전 연구에서 단순히 쌓인 구조의 RNN은 긴 시퀀스에 대해 낮은 재현율을 보이며,   
  이것이 출력 시퀀스를 생성하는데 있어서 latent state의 영향력이 사라지는 것에 있다고 판단합니다.
- 해당 문제를 완화시키기 위해 계층적 RNN 디코더를 사용하는데,   
  입력 시퀀스 $\text{x}$를 $U$개의 서브시퀀스로 나눌 경우에, tanh 활성화 함수가 적용된 fc 레이어를 통과시킨 $z$는 각각의 서브시퀀스와 대응되며,   
  해당 임베딩 벡터 집합 $\text{c}$의 원소가 각각의 fc 레이어를 지나면 $y_u$의 내부 요소에 해당하는 softmax 결과를 생성합니다.
- 위 그림을 참고하여 음악을 예로 들 경우, latent vector가 conductor를 지나면서 마디를 생성하고,   
  각각의 마디에서 RNN 모델을 통해 autoregressive하게 멜로디를 생성하여 전체적으로 latent vector의 영향력이 높아집니다.

##### 3.3. Multi-Stream Modeling
- 음악은 텍스트와 다르게 다양한 악기에 대한 시퀀스가 동시에 주어질 수 있기 때문에,   
  모델링에서 각각의 stream 간 복잡한 의존성을 고려할 필요가 있습니다.
- 3개의 출력 토큰(drum, bass, melody)을 가지는 MusicVAE를 적용하고,   
  계층적 디코더 모델에서는 세 가지 요소를 직교 차원으로 규정하여 각각의 악기에 따라 분리된 디코더 RNN을 사용합니다.
- baseline인 flat 디코더에서는 단일 RNN을 사용하고, 악기 별 softmax를 도출하기 위해 출력 결과를 분리합니다.

#### 5. Experiments

##### 5.1. Data and Training
- 데이터로, 각각의 악기에 대한 노트와 미터 정보가 담긴 MIDI 파일을 사용합니다.
- 학습 데이터로 약 1억 5천만개의 파일을 수집하고, 16마디의 멜로디, 베이스, 드럼 패턴 등을 추출합니다.
- 멜로디의 경우 128개의 피치에 대한 note-on 토큰과 note-off, rest 토큰으로 구성된 130 차원의 벡터 공간으로 구성합니다.
- 드럼 패턴의 경우 61개의 드럼 분류를 9개의 표준 클래스에 맵핑하고 $2^9$개의 범주형 토큰으로 가능한 모든 조합을 표현합니다.
- 모든 모델은 $10^{-3}$에서 $10^{-5}$ 사이의 lr를 적용한 Adam 옵티마이저와 512개의 배치 단위로 학습합니다.

##### 5.2. Short Sequences

<!--<img src="https://d3i71xaburhd42.cloudfront.net/2b050df9e24eb65b0d37f13c6eea1d29b4e316ce/6-Table1-1.png" width="30%" style="margin-left:30px">-->

<table style="text-align:center; margin-left:30px">
  <tr id="head1"><th></th><th colspan="2">Teacher-Forcing</th><th colspan="2">Sampling</th></tr>
  <tr id="head2"><th>Model</th><th>Flat</th><th>Hierarchical</th><th>Flat</th><th>Hierarchical</th></tr>
  <tr id="row1"><td>2-bar Drum</td><td>0.979</td><td>-</td><td>0.917</td><td>-</td></tr>
  <tr id="row2"><td>2-bar Melody</td><td>0.986</td><td>-</td><td>0.951</td><td>-</td></tr>
  <tr id="row3"><td>16-bar Melody</td><td>0.883</td><td>0.919</td><td>0.620</td><td>0.812</td></tr>
  <tr id="row4"><td>16-bar Drum</td><td>0.884</td><td>0.928</td><td>0.549</td><td>0.879</td></tr>
  <tr id="row5"><td>Trio (Melody)</td><td>0.796</td><td>0.848</td><td>0.579</td><td>0.753</td></tr>
  <tr id="row6"><td>Trio (Bass)</td><td>0.829</td><td>0.880</td><td>0.565</td><td>0.773</td></tr>
  <tr id="row7"><td>Trio (Drums)</td><td>0.903</td><td>0.912</td><td>0.641</td><td>0.863</td></tr>
</table>


- 순환 VAE를 통한 음악 시퀀스 생성이 가능함을 증명하기 위해 2마디의 짧은 시퀀스를 활용해 flat 디코더 모델을 학습시켰을 때,   
  입력값에 대해 높은 재현율을 보였지만, 16마디의 긴 시퀀스를 학습시켰을 경우 교사 강요와 샘플링 간 정확도 불일치가 27% 이상 증가합니다.

##### 5.3. Reconstruction Quality
- 계층적 디코더 모델의 경우 NSP 정확도와 노출 편향 가능성을 모두 고려해 학습하였고, 교사 강요 결과와의 불일치를 5-11% 사이로 좁힙니다.
- multi-stream 데이터에 대해서 single-stream으로 활용한 flat 모델과 비교했을 때 압도적인 정확도 향상을 보였고, 교사 강요와도 매우 적은 차이를 보입니다.

#### 6. Conclusion
- MusicVAE 모델을 제안하며, flat baseline보다 긴 시퀀스에서 뿐 아니라 사람을 통한 테스트에서도 높은 성능을 보임을 증명합니다.

<hr style="border-style:dotted">

### TFRecord
- TensorFlow에서 지원하는 파일 형식으로, 데이터를 자체적인 바이너리 형식으로 저장하는 기능을 제공합니다.
- TFRecord 파일 생성은 기록하고자 하는 데이터의 feature를 `tf.train.Example` 객체로 만들어 `tf.io.TFRecordWriter`를 통해 저장합니다.
- `tf.train.Example`에는 `tf.train.BytesList`, `tf.train.FloatList`, `tf.train.IntList` 타입을 담을 수 있으며,   
  문자열을 예로들면 `_bytes_feature()` 함수를 사용해 `tf.train.BytesList` 타입의 `tf.train.Feature` 객체로 변환할 수 있습니다.
- TFRecord의 장점으로는 전처리를 미리 수행할 수 있다는 점으로, 기존의 `from_generator()` 메서드를 이용한 방식보다 향상된 속도를 기대할 수 있습니다.
- TFRecord 파일을 학습을 위한 Dataset으로 정의하기 위해 `tf.data.TFRecordDataset` 모듈을 사용할 수 있습니다.

<br>

```python
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
```

<br>

```python
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()
```

<br>

### MIDI to TFRecord
- Magenta에서는 `scripts.convert_dir_to_note_sequences` 모듈 내 `convert_directory()` 함수를 사용하면,   
  MIDI 파일을 포함하는 전체 디렉토리를 TFRecord 파일로 변환해 저장할 수 있습니다.
- 내부적으로는 `convert_files()` 함수를 통해 `root_dir` 내 모든 파일을 탐색하면서 확장자에 따라 적절한 변환 과정을 수행하며,   
  `recursive=True`를 설정할 경우 `root_dir`의 하위 디렉토리까지 전부 순회합니다.
- MIDI 파일의 경우 `NoteSequence`에서 제공하는 `midi_io` 모듈 내 `midi_to_note_sequence()` 함수를 사용하여,   
  해당하는 파일을 시퀀스로 변환합니다.

<br>

```python
def convert_directory(root_dir, output_file, recursive=False):
  """Converts files to NoteSequences and writes to `output_file`.

  Input files found in `root_dir` are converted to NoteSequence protos with the
  basename of `root_dir` as the collection_name, and the relative path to the
  file from `root_dir` as the filename. If `recursive` is true, recursively
  converts any subdirectories of the specified directory.

  Args:
    root_dir: A string specifying a root directory.
    output_file: Path to TFRecord file to write results to.
    recursive: A boolean specifying whether or not recursively convert files
        contained in subdirectories of the specified directory.
  """
  with tf.io.TFRecordWriter(output_file) as writer:
    convert_files(root_dir, '', writer, recursive)
```

<br>

### References
- https://www.tensorflow.org/tutorials/load_data/tfrecord
- https://hcnoh.github.io/2018-11-05-tensorflow-data-module
- https://velog.io/@riverdeer/TFRecord-파일-읽고-쓰기
- https://github.com/magenta/magenta/blob/main/magenta/scripts/convert_dir_to_note_sequences.py
- https://github.com/magenta/note-seq/blob/main/note_seq/midi_io.py